# Plot results

In [45]:
import os
import numpy as np

# Set parameters
project_root = os.path.realpath(os.pardir)
data_set = 'pixel'
# stimulus_sets = ['jitter', 'static']  # 'static'
start_trial = 1
num_trials = 5
# lambdas = [4, 8, 16]
lambdas = [3, 4, 5, 6, 7, 8]
# sigmas = [2, 4, 8]
bandwidths = np.linspace(1, 1.8, num=3)

epochs = 20
save_loss = 0
data_augmentation = 0
fresh_data = 0
n_gpus = 1

In [46]:
def calc_bandwidth(lambd, sigma):
    r = np.pi*sigma/lambd
    c = np.sqrt(np.log(2)/2)
    return np.log2((r + c)/(r - c))

def calc_sigma(lambd, bandwidth):
    p = 2**bandwidth
    c = np.sqrt(np.log(2)/2)
    return lambd * c / np.pi  * (p + 1) / (p - 1)

In [47]:
# Load accuracy scores and plot
%matplotlib inline
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns


# data_set = 'pixel'
stimulus_set = 'jitter'  # 'static'  # 'jitter'
noise_types = ['Original', 'Salt-and-pepper', 'Additive', 'Single-pixel']  # 'Original'
test_conditions = ['Same', 'Diff', 'NoPix']
results_dir = os.path.join(project_root, 'results', data_set, stimulus_set)

rows = []
test_rows = []

# for stimulus_set in stimulus_sets:
for trial in range(1, 1+num_trials):
    for noise_type in noise_types:
        for lambd in lambdas:
            for bandwidth in bandwidths:
#             for sigma in sigmas:
                sigma = calc_sigma(lambd, bandwidth)

                trial_label = f"{trial}_sigma={sigma:.2}_lambd={lambd}"
                model_name = f"{noise_type}_{trial_label}"
                # print(model_name)

                acc_scores = np.load(os.path.join(results_dir, f'{model_name}_ACC.npy'))
                valacc_scores = np.load(os.path.join(results_dir, f'{model_name}_VALACC.npy'))
                if save_loss:
                    loss = np.load(os.path.join(results_dir, f'{model_name}_LOSS.npy'))
                    valloss = np.load(os.path.join(results_dir, f'{model_name}_VALLOSS.npy'))
                else:
                    loss = np.zeros(epochs)
                    valloss = np.zeros(epochs)

                with open(os.path.join(results_dir, f'{model_name}_CONDVALACC.json'), "r") as jf:
                    cond_acc = json.load(jf)
                if save_loss:
                    with open(os.path.join(results_dir, f'{model_name}_CONDVALLOSS.json'), "r") as jf:
                        cond_loss = json.load(jf)
                else:
                    cond_loss = {condition: 0 for condition in test_conditions}

                for condition in test_conditions:
                    test_rows.append({'Trial': trial, 'Noise Type': noise_type,
                                     'Condition': condition, 'Sigma': sigma, 'Lambda': lambd, 'Bandwidth': bandwidth,
                                      'Loss': cond_loss[condition], 'Accuracy': cond_acc[condition]})
                for epoch in range(epochs):
                    rows.append({'Trial': trial, 'Noise Type': noise_type, 'Sigma': sigma, 'Lambda': lambd, 'Bandwidth': bandwidth,
                                 'Evaluation': 'Testing', 'Epoch': epoch+1, 'Loss': valloss[epoch], 
                                 'Accuracy': valacc_scores[epoch]})

                    rows.append({'Trial': trial, 'Noise Type': noise_type, 'Sigma': sigma, 'Lambda': lambd, 'Bandwidth': bandwidth,
                                 'Evaluation': 'Training', 'Epoch': epoch+1, 'Loss': loss[epoch], 
                                 'Accuracy': acc_scores[epoch]})

scores = pd.DataFrame(rows, columns=['Trial', 'Noise Type', 'Evaluation', 'Sigma', 'Lambda', 'Bandwidth', 'Epoch', 'Loss', 'Accuracy'])
test_scores = pd.DataFrame(test_rows, columns=['Trial', 'Noise Type', 'Condition', 'Sigma', 'Lambda', 'Bandwidth', 'Loss', 'Accuracy'])
# scores
# test_scores

In [ ]:
# g = sns.catplot(x="Condition", y="Accuracy", row='Lambda', col='Sigma', hue="Noise Type", kind="bar", data=test_scores)
g = sns.catplot(x="Noise Type", y="Accuracy", row='Bandwidth', col='Lambda', hue="Condition", kind="bar", data=test_scores)